In [ ]:
# %% [markdown]
# 00 - Data Collection
#
# This notebook handles reading the raw CSVs, validating them, saving copies to `data/raw/`,
# and creating a simple data dictionary and sample rows for inspection.
#
# It references the project specification PDF located at:
# `/mnt/data/2792ae81-88dc-4749-89fb-adeaa7054a3d.pdf`
#
# Usage:
# - Place `application_record.csv` and `credit_record.csv` in the project root or upload them.
# - Run cells sequentially.

# %%
# Standard imports
import os
from pathlib import Path
import pandas as pd
import json

# Paths
PROJECT_ROOT = Path('.').resolve()
RAW_DIR = PROJECT_ROOT / 'data' / 'raw'
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR = PROJECT_ROOT / 'reports' / 'figures'
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Spec PDF path (uploaded spec)
SPEC_PDF_PATH = '/mnt/data/2792ae81-88dc-4749-89fb-adeaa7054a3d.pdf'

print(f"Project root: {PROJECT_ROOT}")
print(f"Spec PDF: {SPEC_PDF_PATH}")
print(f"Raw data directory: {RAW_DIR}")

# %% [markdown]
# Cell: Helper functions
# - save_dataframe_sample: saves a CSV sample for quick viewing
# - create_data_dictionary: builds a very small data dictionary from a dataframe

# %%

def save_dataframe_sample(df: pd.DataFrame, dest: Path, name: str, n: int = 5):
    dest.mkdir(parents=True, exist_ok=True)
    sample_path = dest / f"{name}_sample.csv"
    df.head(n).to_csv(sample_path, index=False)
    print(f"Saved sample to: {sample_path}")
    return sample_path


def create_data_dictionary(df: pd.DataFrame, dest: Path, name: str):
    """Create a simple data dictionary with column, dtype, non-null count, unique sample values."""
    meta = []
    for col in df.columns:
        dtype = str(df[col].dtype)
        non_null = int(df[col].notnull().sum())
        unique_vals = df[col].dropna().unique()[:5].tolist()
        meta.append({
            'column': col,
            'dtype': dtype,
            'non_null': non_null,
            'sample_values': json.dumps(unique_vals, default=str)
        })
    dd = pd.DataFrame(meta)
    dest.mkdir(parents=True, exist_ok=True)
    dd_path = dest / f"{name}_data_dictionary.csv"
    dd.to_csv(dd_path, index=False)
    print(f"Saved data dictionary to: {dd_path}")
    return dd_path

# %% [markdown]
# Cell: Load raw files (attempt to read common locations)
# - Looks for files in project root and in /mnt/data (where uploaded files often appear)

# %%
# Candidate filenames
app_fnames = ['application_record.csv', 'application_record.csv.gz']
credit_fnames = ['credit_record.csv', 'credit_record.csv.gz']

# Candidate dirs
candidate_dirs = [PROJECT_ROOT, Path('/mnt/data'), RAW_DIR]

found_app = None
found_credit = None
for d in candidate_dirs:
    for fn in app_fnames:
        p = d / fn
        if p.exists():
            found_app = p
            break
    for fn in credit_fnames:
        p = d / fn
        if p.exists():
            found_credit = p
            break

print('Found application file:', found_app)
print('Found credit file:', found_credit)

if found_app is None or found_credit is None:
    raise FileNotFoundError(
        'Could not find both application_record.csv and credit_record.csv in project root or /mnt/data.\n'
        'Please upload them to the workspace or place them in the project root.'
    )

# Read files
print('Reading application_record...')
df_app = pd.read_csv(found_app)
print('Reading credit_record...')
df_credit = pd.read_csv(found_credit)

print('\napplication_record shape:', df_app.shape)
print('credit_record shape:', df_credit.shape)

# %% [markdown]
# Cell: Save raw copies into `data/raw/` (keeps originals intact)

# %%
app_dest = RAW_DIR / 'application_record.csv'
credit_dest = RAW_DIR / 'credit_record.csv'

# Only copy if different
if found_app.resolve() != app_dest.resolve():
    df_app.to_csv(app_dest, index=False)
    print(f'Copied application_record to {app_dest}')
else:
    print('Application record already in raw directory')

if found_credit.resolve() != credit_dest.resolve():
    df_credit.to_csv(credit_dest, index=False)
    print(f'Copied credit_record to {credit_dest}')
else:
    print('Credit record already in raw directory')

# %% [markdown]
# Cell: Quick validation checks
# - Check for common ID columns present in both datasets
# - Check for STATUS column in credit_record (required for label creation)

# %%
possible_id_cols = ['ID', 'id', 'client_id', 'Client_id', 'application_id', 'applicationId']
common_cols = set(df_app.columns).intersection(set(df_credit.columns))
print('Common columns between app & credit:', common_cols)

inferred_id = None
for c in possible_id_cols:
    if c in df_app.columns and c in df_credit.columns:
        inferred_id = c
        break

if inferred_id is None:
    # if exactly one common column exists, use it
    if len(common_cols) == 1:
        inferred_id = list(common_cols)[0]
        print(f"Inferred ID column as the single common column: {inferred_id}")
    else:
        print('\nCould not unambiguously infer an ID column.\n')
        print('Please check the column names and set the ID column manually in the next cell.')

print('Inferred ID column:', inferred_id)

# STATUS check
if 'STATUS' not in df_credit.columns:
    print('WARNING: STATUS column not found in credit_record. Label creation will fail until this is resolved.')
else:
    print('STATUS column found in credit_record.')

# %% [markdown]
# Cell: Save sample rows and data dictionaries for both tables

# %%
save_dataframe_sample(df_app, RAW_DIR, 'application_record', n=10)
save_dataframe_sample(df_credit, RAW_DIR, 'credit_record', n=10)
create_data_dictionary(df_app, RAW_DIR, 'application_record')
create_data_dictionary(df_credit, RAW_DIR, 'credit_record')

# %% [markdown]
# Cell: Summary & next steps
# - We have copied raw CSVs to `data/raw/`.
# - Review the inferred ID column above. If it's incorrect, set `INFERRED_ID = '<your_id_col>'` below and re-run the merge notebook.
# - Next: run `01-merge_and_label.ipynb` which will create the label and merged dataset at `/data/processed/merged.csv`.

# %%
# If you need to override the inferred ID, set it here and save for the next notebook
INFERRED_ID = inferred_id
print('Set INFERRED_ID for downstream notebooks:', INFERRED_ID)

# Save a small metadata JSON for the pipeline to read
meta = {
    'spec_pdf': SPEC_PDF_PATH,
    'inferred_id': INFERRED_ID,
    'raw_app_path': str(app_dest),
    'raw_credit_path': str(credit_dest)
}

meta_path = PROJECT_ROOT / 'data' / 'raw' / 'metadata.json'
with open(meta_path, 'w') as f:
    json.dump(meta, f, indent=2)

print(f'Saved metadata to {meta_path}')
